In [9]:
import pandas as pd
import numpy as np
from ofxparse import OfxParser
import matplotlib.pyplot as plt
import os
import tabula
from datetime import datetime

In [103]:
# clean early 18 data 
test = pd.read_pickle("early18.pkl")
mon = {"Jan":1,
    "Feb":2,
    "Mar":3,
    "Apr":4,
    "May":5,
    "June":6,
    "July":7,
    "Aug":8,
    "Sept":9,
    "Oct":10,
    "Nov":11,
    "Dec":12}
test.columns = [x.lower() for x in test.columns]
# test = test.dropna(axis=0)
test.credit = test.credit.str[1:]
test.credit = test.credit.fillna(0)
test = test.fillna("xxx")

In [153]:
# datetime(2018, mon[test.Date[3:]], int(test.Date[:2]))
# test.date = "2018-" + str(mon[test.Date[3:]]) + "-" +  test.Date[:2]


# pos = [24, 175, 191, 194, 206, 242, 275]
# for x in pos:
#     test['credit'].iloc[x-1] = test['credit'].iloc[x]

# test = test.drop([24, 175, 191, 194, 206, 242, 275], axis=0)

test.to_csv("early19.csv")

In [17]:
# get downloaded statments
def get_files(filetype):
    files = []
    for f in os.listdir('statements/'):
        if not f.startswith('.') and ("." + filetype) in f: 
            files.append("statements/" + f)
    
    return files

In [18]:
# get tables from PDF
def get_tables(file, pages="all", multiple_tables=True, debug=True, area=()):
    tables = tabula.read_pdf(file, 
                             multiple_tables=multiple_tables, 
                             pages=pages, 
                             silent=debug  # suppres stdoutput
                             )

    # drop unnamed columns
    for i in range(len(tables)):
        tables[i].fillna("", inplace=True)

        if 'Balance' in tables[i].iloc[0].tolist():
            # swap first row for headers, then drop
            tables[i].columns = [str(i) for i in tables[i].iloc[0]]
            tables[i] = tables[i].iloc[1:]

        # drop empty columns
        tables[i].drop(tables[i].filter(regex='Unnamed|nan').columns, axis=1, inplace=True)



    # join into one table
    frames = [f for f in tables if len(f.columns) == 5]
    table = pd.concat(frames)
    table.columns = [col.lower() for cols in table.columns]

    # clean up the rows
    # table.drop()
    
    
    return table

In [19]:
# returnt he transaction from .ofx file type
def get_transactions(filename):
    with open(filename) as f:
        ofx = OfxParser.parse(f)

    account = ofx.account
    statement = account.statement
    transactions = statement.transactions

    return transactions

In [20]:
# build dataframe
df = pd.DataFrame(columns=['date', 'year', 'month', 'day', 'description', 'debit', 'credit'])

files = get_files("ofx")

for f in files:
    for t in get_transactions(f):
        debit = t.amount if t.type == "debit" else 0
        credit = t.amount if t.type == "credit" else 0
        new = {'date':t.date, 'year': t.date.year, 'month': t.date.month, 'day': t.date.day, 'description': t.memo, 'debit': debit, 'credit': credit}
        
        df = df.append(new, ignore_index=True)

df = df.astype({"description": str, "debit": float, "credit": float})

In [155]:
# save the data frame for analysis
x = pd.read_pickle("./mid18_20.pkl")

In [164]:
y = pd.read_csv("early19.csv")
y

,date,transaction,debit,credit
0,11/1/18,2018 OPENING BALANCE,0.0,0.0
1,18/1/18,POS 15533600 18 JAN,0.0,188.0
2,19/1/18,POS 15533600 19 JAN,0.0,75.0
3,20/1/18,POS 15533600 20 JAN,0.0,359.0
4,21/1/18,POS 15533600 21 JAN,0.0,230.0
...,...,...,...,...
186,17/7/18,POS 15533600 17 JUL,0.0,385.0
187,18/7/18,POS 15533600 18 JUL,0.0,214.0
188,19/7/18,Direct Credit 403697 AMEX,0.0,61.0
189,19/7/18,POS 15533600 19 JUL,0.0,376.0
